In [24]:
from dotenv import load_dotenv
load_dotenv("/home/marshath/play/chainlink/algovate/.env")

Python-dotenv could not parse statement starting at line 4


True

In [25]:
import pickle
import tiktoken
import pandas as pd
import ipywidgets as widgets
from typing import List, Any, Dict
from IPython.display import display, Markdown, clear_output
from langchain.docstore.document import Document
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain, LLMChain
from langchain.retrievers import TFIDFRetriever
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

In [26]:
with open("/home/marshath/play/chainlink/algovate/algovate/data/combined_documents.pkl", "rb") as f:
    documents = pickle.load(f)

In [27]:
class CustomeSplitter:
    def __init__(self, chunk_threshold=6000, chunk_size=6000, chunk_overlap=50):
        self.chunk_threshold = chunk_threshold
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.enc = tiktoken.get_encoding("cl100k_base")
        self.splitter = TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    def token_counter(self, document):
        tokens = self.enc.encode(document.page_content)
        return len(tokens)

    def split(self, documents):
        chunked_documents = []
        for i, doc in enumerate(documents):
            try:
                if self.token_counter(doc) > self.chunk_threshold:
                    chunks = self.splitter.split_documents([doc])
                    chunks = [Document(page_content=chunk.page_content, metadata={"source": f"{chunk.metadata['source']} chunk {i}"}) for i, chunk in enumerate(chunks)]
                    chunked_documents.extend(chunks)
                else:
                    chunked_documents.append(doc)
            except Exception as e:
                chunked_documents.append(doc)
                print(f"Error on document {i}")
                print(e)
                print(doc.metadata["source"])

        return chunked_documents

In [28]:
splitter = CustomeSplitter()
chunked_documents = splitter.split(documents)

In [29]:
template = """
You are an AI assistant helping to label data.
Given a text document, you need to find if the document contains code or not.
If the document contains code, you need to find the language of the code.
Give your answer in the form of a list of dictionaries.

Example:
```
{{"code": True, "language": "python"}}
```

Everythin between the triple quotes is code has to be a valid python list of dictionaries. 
"""
human_template = """{text}"""
prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessagePromptTemplate.from_template(template),
            HumanMessagePromptTemplate.from_template(human_template),
        ]
    )


In [58]:
i = 5
chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo-16k"), prompt=prompt)
res = chain.predict(text=chunked_documents[i])
print(res)
Markdown(chunked_documents[i].page_content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


[{"code": True, "language": "solidity"}]


# Existing Job Request

  * Overview
  * Example

Using an _existing_ Oracle Job makes your smart contract code more succinct.
This page explains how to retrieve the gas price from an existing Chainlink
job that calls [etherscan gas tracker API](https://docs.etherscan.io/api-endpoints/gas-tracker#get-gas-oracle).

Prerequisites

You should be familiar with the [Chainlink Basic Request Model](/architecture-overview/architecture-request-model/). If you are new to developing smart
contracts on Ethereum, see the [Getting Started](/getting-started/conceptual-overview/) guide to learn the basics.

## Example

In [Single Word Response Example](/any-api/get-request/examples/single-word-response), the example contract code declared which URL to use, where to find
the data in the response, and how to convert it so that it can be represented
on-chain.

This example uses an existing job that is pre-configured to make requests to
get [the gas price](https://docs.etherscan.io/api-endpoints/gas-tracker#get-gas-oracle). Using specialized jobs makes your contracts succinct and more
simple.

[Etherscan gas oracle](https://docs.etherscan.io/api-endpoints/gas-tracker#get-gas-oracle) returns the current Safe, Proposed and Fast gas
prices. To check the response, you can directly paste the following URL in
your browser
`https://api.etherscan.io/api?module=gastracker&action=gasoracle&apikey=YourApiKeyToken`
or run this command in your terminal:


    curl -X 'GET' \
      'https://api.etherscan.io/api?module=gastracker&action=gasoracle&apikey=YourApiKeyToken' \
      -H 'accept: application/json'

The response should be similar to the following:


    {
      "status": "1",
      "result": {
        "LastBlock": "14653286",
        "SafeGasPrice": "33",
        "ProposeGasPrice": "33",
        "FastGasPrice": "35",
        "suggestBaseFee": "32.570418457",
        "gasUsedRatio": "0.366502543599508,0.15439818258491,0.9729006,0.4925609,0.999657066666667"
      }
    }

For this example, we created a job that leverages the [EtherScan External
Adapter](https://github.com/smartcontractkit/external-adapters-js/tree/develop/packages/sources/etherscan) to fetch the _SafeGasPrice_ ,
_ProposeGasPrice_ and _FastGasPrice_. You can learn more about External
Adapters [here](/chainlink-nodes/external-adapters/external-adapters). To
consume an API, your contract must import
[ChainlinkClient.sol](https://github.com/smartcontractkit/chainlink/blob/master/contracts/src/v0.8/ChainlinkClient.sol).
This contract exposes a struct named `Chainlink.Request`, which your contract
can use to build the API request. The request must include the following
parameters:

  * Link token address
  * Oracle address
  * Job id
  * Request fee
  * Task parameters
  * Callback function signature

Note on Funding Contracts

Making a GET request will fail unless your deployed contract has enough LINK
to pay for it. **Learn how to[Acquire testnet LINK](/resources/acquire-link)
and [Fund your contract](/resources/fund-your-contract)**.


    // SPDX-License-Identifier: MIT
    pragma solidity ^0.8.7;
    
    import "@chainlink/contracts/src/v0.8/ChainlinkClient.sol";
    import "@chainlink/contracts/src/v0.8/ConfirmedOwner.sol";
    
    /**
     * Request testnet LINK and ETH here: https://faucets.chain.link/
     * Find information on LINK Token Contracts and get the latest ETH and LINK faucets here: https://docs.chain.link/docs/link-token-contracts/
     */
    
    /**
     * THIS IS AN EXAMPLE CONTRACT THAT USES HARDCODED VALUES FOR CLARITY.
     * THIS IS AN EXAMPLE CONTRACT THAT USES UN-AUDITED CODE.
     * DO NOT USE THIS CODE IN PRODUCTION.
     */
    
    contract GetGasPrice is ChainlinkClient, ConfirmedOwner {
        using Chainlink for Chainlink.Request;
    
        uint256 public gasPriceFast;
        uint256 public gasPriceAverage;
        uint256 public gasPriceSafe;
    
        bytes32 private jobId;
        uint256 private fee;
    
        event RequestGasPrice(
            bytes32 indexed requestId,
            uint256 gasPriceFast,
            uint256 gasPriceAverage,
            uint256 gasPriceSafe
        );
    
        /**
         * @notice Initialize the link token and target oracle
         *
         * Sepolia Testnet details:
         * Link Token: 0x779877A7B0D9E8603169DdbD7836e478b4624789
         * Oracle: 0x6090149792dAAeE9D1D568c9f9a6F6B46AA29eFD (Chainlink DevRel)
         * jobId: 7223acbd01654282865b678924126013
         *
         */
        constructor() ConfirmedOwner(msg.sender) {
            setChainlinkToken(0x779877A7B0D9E8603169DdbD7836e478b4624789);
            setChainlinkOracle(0x6090149792dAAeE9D1D568c9f9a6F6B46AA29eFD);
            jobId = "7223acbd01654282865b678924126013";
            fee = (1 * LINK_DIVISIBILITY) / 10; // 0,1 * 10**18 (Varies by network and job)
        }
    
        /**
         * Create a Chainlink request the gas price from Etherscan
         */
        function requestGasPrice() public returns (bytes32 requestId) {
            Chainlink.Request memory req = buildChainlinkRequest(
                jobId,
                address(this),
                this.fulfill.selector
            );
            // No need extra parameters for this job. Send the request
            return sendChainlinkRequest(req, fee);
        }
    
        /**
         * Receive the responses in the form of uint256
         */
        function fulfill(
            bytes32 _requestId,
            uint256 _gasPriceFast,
            uint256 _gasPriceAverage,
            uint256 _gasPriceSafe
        ) public recordChainlinkFulfillment(_requestId) {
            emit RequestGasPrice(
                _requestId,
                _gasPriceFast,
                _gasPriceAverage,
                _gasPriceSafe
            );
            gasPriceFast = _gasPriceFast;
            gasPriceAverage = _gasPriceAverage;
            gasPriceSafe = _gasPriceSafe;
        }
    
        /**
         * Allow withdraw of Link tokens from the contract
         */
        function withdrawLink() public onlyOwner {
            LinkTokenInterface link = LinkTokenInterface(chainlinkTokenAddress());
            require(
                link.transfer(msg.sender, link.balanceOf(address(this))),
                "Unable to transfer"
            );
        }
    }
    

To use this contract:

  1. Open the [contract in Remix](https://remix.ethereum.org/#url=https://docs.chain.link/samples/APIRequests/GetGasPrice.sol).

  2. Compile and deploy the contract using the Injected Provider environment. The contract includes all the configuration variables for the _Sepolia_ testnet. Make sure your wallet is set to use _Sepolia_. The _constructor_ sets the following parameters:

    * The Chainlink Token address for _Sepolia_ by calling the [`setChainlinkToken`](/any-api/api-reference/#setchainlinktoken) function.
    * The Oracle contract address for _Sepolia_ by calling the [`setChainlinkOracle`](/any-api/api-reference/#setchainlinkoracle) function.
    * The `jobId`: A specific job for the oracle node to run. In this case, the job is very specific to the use case as it returns the gas prices. You can find the job spec for the Chainlink node [here](/chainlink-nodes/job-specs/direct-request-existing-job).
  3. Fund your contract with 0.1 LINK. To learn how to send LINK to contracts, read the [Fund Your Contracts](/resources/fund-your-contract) page.

  4. Call the `gasPriceFast`, `gasPriceAverage` and `gasPriceSafe` functions to confirm that the `gasPriceFast`, `gasPriceAverage` and `gasPriceSafe` state variables are equal to zero.

  5. Run the `requestGasPrice` function. This builds the `Chainlink.Request`. Note how succinct the request is.

  6. After few seconds, call the `gasPriceFast`, `gasPriceAverage` and `gasPriceSafe` functions. You should get a non-zero responses.

### Response Types

Make sure to choose an oracle job that supports the data type that your
contract needs to consume. Multiple data types are available such as:

  * **`uint256`** \- Unsigned integers
  * **`int256`** \- Signed integers
  * **`bool`** \- True or False values
  * **`string`** \- String
  * **`bytes32`** \- Strings and byte values. If you need to return a string, use `bytes32`. Here's [one method](https://gist.github.com/alexroan/a8caf258218f4065894ecd8926de39e7) of converting `bytes32` to `string`. Currently, any return value must fit within 32 bytes. If the value is bigger than that, make multiple requests.
  * **`bytes`** \- Arbitrary-length raw byte data

## Setting the LINK token address, Oracle, and JobId

The [`setChainlinkToken`](/any-api/api-reference/#setchainlinktoken) function
sets the LINK token address for the [network](/resources/link-token-contracts/) you are deploying to. The [`setChainlinkOracle`](/any-api/api-reference/#setchainlinkoracle) function sets a specific Chainlink oracle that
a contract makes an API call from. The `jobId` refers to a specific job for
that node to run.

Each job is unique and returns different types of data. For example, a job
that returns a `bytes32` variable from an API would have a different `jobId`
than a job that retrieved the same data, but in the form of a `uint256`
variable.

Check the [Find Existing Jobs page](/any-api/find-oracle/) to learn how to
find a job suitable to your use case.

## What's next

  * [› Find Existing Jobs](/any-api/find-oracle/)
  * [› API Reference](/any-api/api-reference/)
  * [› Testnet Oracles](/any-api/testnet-oracles/)

  * [ Edit this page ](https://github.com/smartcontractkit/documentation/tree/main/src/pages/any-api/get-request/examples/existing-job-request.mdx)
  * [ Join our community ](https://discord.com/invite/aSK4zew)

